Nesta prática você irá implementar o indexador para, logo após, indexar o conteúdo da Wikipédia. Nesta prática, o índice é composto pela classe abstrata `Index` que armazena a estrutura do índice e possui as operações básicas do mesmo. 

Iremos fazer duas implementações desse índice: o `HashIndex` que será um índice simples em memória principal e o `FileIndex` em que as ocorrências ficarão em memória secundária para possibilitar a indexação de uma quantidade maior de páginas. Assim, teremos os seguintes arquivos:

- `structure.py`: Possui toda a estrutura do índice;
- `index_structure_test.py`: Testa a estrutura do índice;
- `file_index_test.py`: Possui os testes unitários específicos para a indexação das ocorrencias em arquivos da classe `FileIndex`;
- `performance_test.py`: Executa um teste de performance (tempo de execução e memória utilizada) do índice;
- `indexer.py`: Possui as classes para o preprocessamento e preparação para a indexação;
- `indexer_test.py`: Realiza o [teste de integração](https://en.wikipedia.org/wiki/Integration_testing) da prática como um todo (inclusive as funções da indexer.py).

Na entrega, não esqueça de apresentar a saída de execução de cada atividade desta tarefa.

## Implementação da classe Abstrata `Index` e classe `HashIndex`

A classe abstrata Index, no arquivo `structure.py` possui os seguintes atributos:

- `dic_index`:  dicionário em que a chave é o termo indexação (string, gerenciado por esta classe) e, os valores,  podem der de diferentes tipos - dependendo da subclasse;
- `set_documents`: conjunto de ids de documentos existentes.

Os métodos serão discutidos ao longo das atividades. Inicialmente, iremos fazer a importação do módulo:

In [1]:
from index.structure import *

**Atividade 1 - método index da classe Index**: Este método esta quase todo pronto e é reponsável por indexar um termo com sua frequncia e documento no índice, de acordo com uma de suas subclasses. Você deve deve apenas inicializar a variável `int_term_id` apropriadamente - substituindo os `None` correspondente. Caso o termo não exista no índice, deverá obter o próximo term_id. Esse id pode ser sequencial. Caso esse id seja encontrado, a classe Index deverá chamar o método `get_term_id` (implementado pelas subclasses) para obtê-lo pois, dependendo da implementação, haverá uma forma diferente de obtenção. Além disso, você deverá atualizar o atributo `set_documents` apropriadamente. Para testar tanto esta atividade e a seguinte,  você deverá fazer uma das implementações dessa classe - a classe **HashIndex** - na atividade 3. 

**Atividade 2 - atributos calculados da classe Index**: Você deve implementar os atributos calculados `document_count` e `vocabulary` da classe Index. O atributo `document_count` retorna a quantidade de documentos existentes e, `vocabulary` retorna uma lista com o vocabulário completo indexado. 

**Atividade 3 - Implementação da classe HashIndex: ** O HashIndex deverá fazer um índice em memória.

Como exemplo, caso tenhamos três documentos $d_1 = $"A casa verde é uma casa bonita", $d_2 = $"A casa bonita" e $d_3 =$"O prédio verde", caso não haja remoção de _stopwords_ nem acentos, o atributo `dic_index` deverá possuir a seguinte estrutura:

In [2]:
{"a": [TermOccurrence(1,1,1), TermOccurrence(2,1,1)],
 "casa": [TermOccurrence(1,2,2), TermOccurrence(2,2,1)],
 "verde": [TermOccurrence(1,3,1), TermOccurrence(3,3,1)],
 "é": [TermOccurrence(1,4,1)],
 "uma": [TermOccurrence(1,5,1)],
 "bonita": [TermOccurrence(1,6,1),TermOccurrence(2,6,1)],
 "o": [TermOccurrence(3,7,1)],
 "prédio": [TermOccurrence(3,8,1)]
}

{'a': [(term_id:1 doc: 1 freq: 1), (term_id:1 doc: 2 freq: 1)],
 'casa': [(term_id:2 doc: 1 freq: 2), (term_id:2 doc: 2 freq: 1)],
 'verde': [(term_id:3 doc: 1 freq: 1), (term_id:3 doc: 3 freq: 1)],
 'é': [(term_id:4 doc: 1 freq: 1)],
 'uma': [(term_id:5 doc: 1 freq: 1)],
 'bonita': [(term_id:6 doc: 1 freq: 1), (term_id:6 doc: 2 freq: 1)],
 'o': [(term_id:7 doc: 3 freq: 1)],
 'prédio': [(term_id:8 doc: 3 freq: 1)]}

Por simplicidade deste indice, perceba que deixamos o `term_id` de forma repetida. Iremos deixar assim, porém poderiamos retirar essa redundancia para reduzir o consumo de memória. Porém, nesta prática, iremos implementar o índice em arquivo que irá ser melhor ainda na questão de consumo de memória ;)

O índice é chamado da seguinte forma - esse código só ira funcionar depois que vocẽ terminar esta atividade :):

In [3]:
index = HashIndex()
#indexação do documento 1
index.index("a",1,1)
index.index("casa",1,2)
index.index("verde",1,1)
index.index("é",1,1)
index.index("uma",1,1)
#indexação do documento 2
index.index("a",2,1)
index.index("casa",2,1)
index.index("bonita",2,1)

#indexação do documento 3
index.index("o",3,1)
index.index("prédio",3,1)
index.index("verde",3,1)

index.finish_indexing()

A classe `Index` é que irá manter o dicinário com o vocabulário do índice e, dependendo de sua implementação, suas ocorrencias. Iremos fazer duas implementações: a classe `HashIndex`, que faz o indice e suas ocorrencias em memória principal e a classe `FileIndex`, que armazena as ocorrências em arquivo, ambas subclasses de `Index`. O método `finish_indexing` é um método que não está implementado no `Index` e é implementado (opcionalmente) nas suas subclasses caso haja necessidade de fazer algo no final da indexação. Em nosso caso, apenas a classe `FileIndex` irá precisar.

Agora, você deverá implementar a classe HashIndex. Para sua implementação, você deverá completar os seguintes métodos/atributo calculados:
- `create_index_entry`: cria uma nova entrada no índice utilizando, se necessário, o id do termo passado como parâmetro - não será necessário agora. A implementação deste método é **super simples** - apenas substitua o None. Verifique também o método `index` da superclasse para entender melhor o que será retornado;
- `add_index_occur`: Adiciona uma nova ocorrencia na entrada deste índice. Você precisará da entrada do termo atual, o id do documento e frequencia do termo no documento, passado como parâmetro. Atualize substituindo os `None`. Veja também o modo `index` da superclasse para entender melhor como o método funciona.

Faça os testes baixo para garantir que a atividade atual e as duas anteriores foram implementadas corretamente. Nos primeiros dois testes, você ainda não verá a lista de ocorrências, pois o método para obtê-la será implementado a seguir. 

In [4]:
!python3 -m index.index_structure_test StructureTest.test_vocabulary

======= Indice Gerado ======
casa -> [(term_id:0 doc: 1 freq: 10), (term_id:0 doc: 2 freq: 3)]
vermelho -> [(term_id:1 doc: 1 freq: 3), (term_id:1 doc: 2 freq: 1), (term_id:1 doc: 3 freq: 1)]
verde -> [(term_id:2 doc: 1 freq: 1)]
.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


In [5]:
!python3 -m index.index_structure_test StructureTest.test_document_count

======= Indice Gerado ======
casa -> [(term_id:0 doc: 1 freq: 10), (term_id:0 doc: 2 freq: 3)]
vermelho -> [(term_id:1 doc: 1 freq: 3), (term_id:1 doc: 2 freq: 1), (term_id:1 doc: 3 freq: 1)]
verde -> [(term_id:2 doc: 1 freq: 1)]
.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


Agora, implemente os métodos:

- `get_occurrence_list`: Retornará a lista de ocorrencias de um determinado termo. Considerando o exemplo apresentado no inicio desta atividade, `index.get_occurrence_list('casa')` retornará a lista `[TermOccurrence(1,2,2), TermOccurrence(2,2,1)]`. Caso um termo não exista, este método deverá retornar uma lista vazia. Logo após, execute o teste abaixo:

In [6]:
!python3 -m index.index_structure_test StructureTest.test_get_occurrence_list

======= Indice Gerado ======
casa -> [(term_id:0 doc: 1 freq: 10), (term_id:0 doc: 2 freq: 3)]
vermelho -> [(term_id:1 doc: 1 freq: 3), (term_id:1 doc: 2 freq: 1), (term_id:1 doc: 3 freq: 1)]
verde -> [(term_id:2 doc: 1 freq: 1)]
.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


- `document_count_with_term`: Retorna a quantidade de documentos que possuem um determinado termo. Considerando o exemplo apresentado no inicio desta atividade, `index.document_count_with_term('casa')` retornará 2. Caso um termo não exista, este método deverá retornar zero. Logo após, execute o teste abaixo:

In [7]:
!python3 -m index.index_structure_test StructureTest.test_document_count_with_term

======= Indice Gerado ======
casa -> [(term_id:0 doc: 1 freq: 10), (term_id:0 doc: 2 freq: 3)]
vermelho -> [(term_id:1 doc: 1 freq: 3), (term_id:1 doc: 2 freq: 1), (term_id:1 doc: 3 freq: 1)]
verde -> [(term_id:2 doc: 1 freq: 1)]
.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


**Atividade 4 - métodos de comparação da classe TermOccurrence**: Eventualmente iremos precisar ordenar as ocorrências Por isso, temos que implementar os [comparadores de `__eq__` e `__lt__`](https://docs.python.org/3.7/reference/datamodel.html#object.__lt__) além de usar o _decorator_ [total_ordering](https://docs.python.org/3.7/library/functools.html#functools.total_ordering) - [veja também aqui](https://portingguide.readthedocs.io/en/latest/comparisons.html#rich-comparisons). `__eq__` retorna igual se um objeto é considerado igual ao outro. Considere que uma ocorrencia é igual a outra se o id do termo dela e o id do documento forem iguais. 

O comparador `<` é implementado pelo método `__lt__` que retorna verdadeiro sde o objeto corrrente `self` é menor do que o objeto passado como parametro. A ocorrencia deverá ser ordenada primeiramente pelo seu `term_id` e, logo após, pelo `doc_id`. Faça o exemplo abaixo para testar (não esqueça de reiniciar o Kernel quando modificar o código):

In [8]:
from index.structure import *
t1 = TermOccurrence(1,1,2)
t2 = TermOccurrence(3,1,2)
t3 = TermOccurrence(1,2,2)
t4 = TermOccurrence(2,2,2)
t5 = TermOccurrence(2,2,2)


print(f"Resultado obtido: {t1 == t5} - esperado: False")
print(f"Resultado obtido: {t4 == t5} - esperado: True")
print(f"Resultado obtido: {t1 != t1} - esperado: False")
print(f"Resultado obtido: {t1 == None} - esperado: False")
print(f"Resultado obtido: {t1 < t2} - esperado: True")
print(f"Resultado obtido: {t2 > t3} - esperado: False")
print(f"Resultado obtido: {t3 < t4} - esperado: True")
print(f"Resultado obtido: {t2 > t4} - esperado: False")
print(f"Resultado obtido: {t2 > None} - esperado: False")

Resultado obtido: False - esperado: False
Resultado obtido: True - esperado: True
Resultado obtido: False - esperado: False
Resultado obtido: False - esperado: False
Resultado obtido: True - esperado: True
Resultado obtido: False - esperado: False
Resultado obtido: True - esperado: True
Resultado obtido: False - esperado: False
Resultado obtido: False - esperado: False


## Construção de índice usando arquivo

A construção de índice usando apenas memória principal fácil de implementar e eficiente em termos de tempo de execução. Porém, quando precisamos de indexar milhões/bilhões de páginas, é muitas vezes inviável armazenarmos tudo em memória principal. 

Para resolver esse problema, uma solução é mantermos o vocabulário em memória principal e as ocorrências em memória secundária. Assim, teriamos o mesmo atributo `dic_index` na classe `Index`. Porém, cada entrada (termo) referenciará as ocorrencias em arquivo. Utilizando exemplo da atividade 3 neste contexto, no final da indexação, o `dic_index` deve ficar da seguinte forma: 

In [9]:
{"a": TermFilePosition(1, 0, 2), 
 "casa": TermFilePosition(2, 20, 2), 
 "verde": TermFilePosition(3, 40, 2),
 "é": TermFilePosition(4, 60, 1), 
 "uma": TermFilePosition(5, 70, 1), 
 "bonita": TermFilePosition(6, 80, 2), 
 "o": TermFilePosition(7, 100, 1), 
 "prédio": TermFilePosition(8, 110, 1), 
}




{'a': term_id: 1, doc_count_with_term: 2, term_file_start_pos: 0,
 'casa': term_id: 2, doc_count_with_term: 2, term_file_start_pos: 20,
 'verde': term_id: 3, doc_count_with_term: 2, term_file_start_pos: 40,
 'é': term_id: 4, doc_count_with_term: 1, term_file_start_pos: 60,
 'uma': term_id: 5, doc_count_with_term: 1, term_file_start_pos: 70,
 'bonita': term_id: 6, doc_count_with_term: 2, term_file_start_pos: 80,
 'o': term_id: 7, doc_count_with_term: 1, term_file_start_pos: 100,
 'prédio': term_id: 8, doc_count_with_term: 1, term_file_start_pos: 110}

e as ocorrencias, ordenadas por termo e, logo após, por documento ficariam em um arquivo na seguinte ordem:

In [10]:
[TermOccurrence(1,1,1), 
 TermOccurrence(2,1,1), 
 TermOccurrence(1,2,2), 
 TermOccurrence(2,2,1),
 TermOccurrence(1,3,1), 
 TermOccurrence(3,3,1),
 TermOccurrence(1,4,1),
 TermOccurrence(1,5,1),
 TermOccurrence(1,6,1),
 TermOccurrence(2,6,1),
 TermOccurrence(3,7,1),
 TermOccurrence(3,8,1)]

[(term_id:1 doc: 1 freq: 1),
 (term_id:1 doc: 2 freq: 1),
 (term_id:2 doc: 1 freq: 2),
 (term_id:2 doc: 2 freq: 1),
 (term_id:3 doc: 1 freq: 1),
 (term_id:3 doc: 3 freq: 1),
 (term_id:4 doc: 1 freq: 1),
 (term_id:5 doc: 1 freq: 1),
 (term_id:6 doc: 1 freq: 1),
 (term_id:6 doc: 2 freq: 1),
 (term_id:7 doc: 3 freq: 1),
 (term_id:8 doc: 3 freq: 1)]

em que cada instancia da classe `TermFilePosition` é a especificação da posição inicial de um `term_id` em um arquivo  além de especificar também a quantidade de ocorrencias desse termo. Essa posição inicial e quantidade são definidas nos atributos atributos `term_file_start_pos` e `doc_count_with_term`, respectivamente. A posição inicial está em bytes e, nesse exemplo, foi considerado que cada TermOcurrence possui 10 bytes. Assim, por exemplo, o termo casa (term_id=2) inicia-se na posição 20 e possui duas ocorrências. Com isso, é possível obter todas as ocorrencias de um determinado termo.

Para deixarmos a estrutura dessa forma, temos um dificultador: no arquivo, temos que ordenar as ocorrencias pelo termo, porém, indexamos por documento (veja na atividade 3). Assim, se gravássemos as ocorrências assim que indexarmos, as gravariamos agrupadas por documento.

Assim, temos que garantir uma ordenação por termo do arquivo externo, lembrando que nem sempre é possível armazenar todo o arquivo em memória principal. Para resolvermos isso, faremos o seguinte:

- Sempre, ao indexar, salvaremos o indice em uma lista (temporária) de ocorrencia de termos `lst_occurrences_tmp`
- Usaremos o método `save_tmp_occurrences` para, assim que a lista estiver com um determinado tamanho, ordená-la pelo termo e salvar de formar ordenada em um novo arquivo de indice com a todas as ocorrencias. Para que seja feito isso, você deverá fazer uma ordenação externa lendo m consideração o índice em arquivo atual e a lista de ocorrencias temporárias. Segue o passo a passo:
    - (1) ordene a lista `lst_occurrences_tmp`. Lembre-se que você implementou os comparadores das instancias TermOccurrence, assim, a ordenação e descobrir o menor valor entre as ocorrencias é uma operação simples;
    - (2) criar um arquivo novo;
    - (3) compare a primeira posição da lista com a primeira posição do arquivo de índice, sempre inserindo a ocorrencia considerada com o menor entre elas no novo arquivo. Lembrando novamente que os comparadores foram implementados e que você possui os métodos `next_from_list` e `next_from_file` - que será implementado na atividade 5(b) para ajudar;
    - (4) esse novo arquivo passará a ser o índice. Exclua o indice antigo e limpe a lista de ocorrencias `lst_occurrences_tmp`.
- O método `finish_indexing` é o método que será chamado ao finalizar a indexação. Neste contexto ele será usado para organizar o `dic_index` atualizando os atributos `term_file_start_pos` e `doc_count_with_term` para os valores corretos.

Na primeira execução, não haverá arquivo e você adicionará a lista toda no arquivo de forma sequencial. Fazendo esse procedimento, você sempre irá garantir um arquivo ordenado da forma esperada.

**Atividade 5(a) - método de escrita no arquivo:** Iremos necessitar, em algum momento, a escrita de uma instancia de `TermOccurrence` em arquivo. Assim deveremos implementar o método de escrita em arquivo nessa classe. Para economizar espaço e por simplicidade, será escrito em um arquivo binário armazenando os três atributos inteiros. Cada inteiro será armazenado em 4 bytes - será o suficiente para a nossa indexação. Veja um exemplo abaixo de escrita, leitura e impressão do posicionamento no arquivo (esses métodos serão uteis nessa e nas próximas atividades).

In [11]:
x = 100
with open("xuxu.idx","wb") as file:
    print(file.tell())
    file.write(x.to_bytes(4,byteorder="big"))
    print(file.tell())
with open("xuxu.idx","rb") as file:
    print(f"número: {int.from_bytes(file.read(4),byteorder='big')}")

0
4
número: 100


**Atividade 5(b) - métodos next_from_file e next_from_list**: Antes de fazer a ordenação, é interessante implementar esses dois métodos que irão auxiliar na obtenção do menor elemento no arquivo e na lista, respectivamente. Considerando que a lista e o arquivo estão ordenados de forma crescente, temos que obter o primeiro elemento da lista e retirá-lo (utilize o [método pop](https://docs.python.org/3.1/tutorial/datastructures.html)).  

Para a leitura do arquivo, iremos usar a API [pickle](https://docs.python.org/3/library/pickle.html) que facilita inserir/carregar estruturas em arquivo binário. O método `next_from_file` já possui um arquivo aberto e você deverá ler a próxima entrada por meio da função `load` da API `pickle`. Caso não exista próximo elemento, é lançado uma exceção. Em ambos os métodos, caso não exista próximo elemento, será retonado `None`. 

Complete a implementação substituindo os `None` quando julgar necessário.

**Atividade 7 - método save_tmp_occurrences: ** Implemente o método `save_tmp_occurrences`. Esse método deverá salvar a lista `lst_occurrences_tmp` em arquivo de forma ordenada, conforme explicado anteriormente. Neste método, você não precisa preocupar com o atributo `dic_index`. Leve em consideração os seguintes atributos/métodos:

- `idx_file_counter`:  No código, você irá criar sempre novos indices, excluindo o antigo. Este atributo será útil para definirmos o nome do arquivo do índice. O novo arquivo do índice chamará `occur_index_X` em que $X$ é o número do mesmo. 
- `str_idx_file_name`: Atributo que armazena o arquivo indice atual. A primeira vez que executarmos `save_tmp_occurrences` não haverá arquivo criado e, assim `str_idx_file_name = None`
- `lst_occurrences_tmp`: Lista de ocorrencias a serem armazenadas em arquivo
- `next_from_file` e `next_from_list`: implementados na atividade anterior, para obter o proximo item do arquivo ou da lista. Importantes para fazer a ordenação.



Execute o teste unitário abaixo para verificar corretude deste código:

In [12]:
!python3 -m index.file_index_test FileIndexTest.test_save_tmp_occurrences

Primeira execução (criação inicial do indice) [ok]
Inserção de alguns itens - teste 1/2 [ok]
Inserção de alguns itens - teste 2/2 [ok]
.
----------------------------------------------------------------------
Ran 1 test in 0.017s

OK


**Atividade 8: método finish_indexing: ** Agora, com as ocorrências organizadas no arquivo por termo, você deverá implementar o método `finish_indexing` para atualizar o atributo `dic_index` com a posição inicial e quantidade de documentos de cada termo nas suas instancias `TermFilePosition`. Logo após, execute o teste unitário abaixo.

In [13]:
!python3 -m index.file_index_test FileIndexTest.test_finish_indexing

Lista de ocorrências a serem testadas:
(term_id:1 doc: 1 freq: 3)
(term_id:1 doc: 2 freq: 2)
(term_id:1 doc: 3 freq: 1)
(term_id:2 doc: 1 freq: 1)
(term_id:2 doc: 2 freq: 1)
(term_id:2 doc: 3 freq: 2)
(term_id:3 doc: 1 freq: 3)
(term_id:4 doc: 1 freq: 5)
(term_id:4 doc: 2 freq: 5)
Tamanho de cada ocorrência: 94 bytes
.
----------------------------------------------------------------------
Ran 1 test in 0.006s

OK


**Atividade 9 - implementação em `FileIndex` dos métodos abstratos da classe Index: ** Como vocês perceberam, `FileIndex` é subclasse de `Index`. Assim,  precisamos implementar os métodos abstratos da classe `Index`:

- `create_index_entry`: no `FileIndex` para criar uma nova entrada no índice, você deverá retornar uma instancia de `TermFilePosition` para este novo `term_id`. Ao criá-lo, você não precisa de definir a posição inicial do arquivo nem a quantidade de documentos. Conforme vocês implementaram nas atividades anteriores, isso é feito apenas no momento de finalização da indexação;
- `add_index_occur`: Neste caso, você deverá criar e adicionar uma nova ocorrencia na lista de ocorrencias temporárias `lst_occurrences_tmp` e, caso senha passado o limite `FileIndex.TMP_OCCURRENCES_LIMIT` do número máximo de ocorrencias na lista, chamar o método `save_tmp_occurrence`.
- `get_occurrence_list` e `document_count_with_term`: Possuem as mesmas funcionalidades descritas na atividade 3, porém, agora você deverá considerar a estrutura criada no `FileIndex`. Lembrem-se que esse método é só chamado após a finalização da indexação, assim, considere que o índice já está pronto.

**Atividade 10 - Teste unitário** Dessa vez, você deverá alterar uma classe de teste unitário para conseguir executá-la. 

Agora iremos testar os métodos get_occurrence_list e document_count_with_term. Lembre-se que já temos um teste unitário para isso, porém ele testa a estrutura de um `HashIndex`. Neste caso, iremos apenas mudar a estrutura, mas os métodos serão o mesmo, por isso, conseguiremos reaproveitar o teste feito anteriormente criando apenas uma nova classe de teste. 

Implementando este teste, você perceberá como é lindo usar orientação objetos ao seu favor 🥰. No arquivo `index_structure_test.py` você possui a classe `FileStructureTest` que é subclasse de nosso teste criado `StructureTest`.  Você deverá implementar o método `setUp` na classe `FileStructureTest` que sobrepõe o método de mesmo nome na classe `StructureTest`. O método `setUp` é executado sempre antes do teste. Este método que você irá criar irá fazer exatamente a mesma coisa que o criado em `StructureTest` porém, você deverá instanciar um `FileIndex` ao invés de um `HashIndex`. Logo após, execute os testes:

In [14]:
!python3 -m index.index_structure_test  FileStructureTest.test_document_count_with_term

======= Indice Gerado ======
casa -> [(term_id:0 doc: 1 freq: 10), (term_id:0 doc: 2 freq: 3)]
vermelho -> [(term_id:1 doc: 1 freq: 3), (term_id:1 doc: 2 freq: 1), (term_id:1 doc: 3 freq: 1)]
verde -> [(term_id:2 doc: 1 freq: 1)]
.
----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


In [15]:
!python3 -m index.index_structure_test  FileStructureTest.test_get_occurrence_list

======= Indice Gerado ======
casa -> [(term_id:0 doc: 1 freq: 10), (term_id:0 doc: 2 freq: 3)]
vermelho -> [(term_id:1 doc: 1 freq: 3), (term_id:1 doc: 2 freq: 1), (term_id:1 doc: 3 freq: 1)]
verde -> [(term_id:2 doc: 1 freq: 1)]
.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


De forma similar, também criamos um teste de performance. Verifique a performance da indexação em arquivo e em memória ao indexar milhões de ocorrencias de termos utilizando os testes abaixo. Note que, desta vez, estamso chamando os testes por meio de comandos Python e não pelo terminal. Assim, caso queira fazer alguma alteração no arquivos `.py`, você deve reiniciar o kernel.

- Índice completamente em memória principal:

In [16]:
import unittest
from index.performance_test import PerformanceTest

PerformanceTest.NUM_DOCS = 2500
PerformanceTest.NUM_TERM_PER_DOC = 500

suite = unittest.TestLoader().loadTestsFromTestCase(PerformanceTest)
unittest.TextTestRunner(verbosity=2).run(suite)

Memoria usada: 203.545836 MB; Máximo 203.54598 MB
Indexando ocorrencia #1,250,000/1,250,000 (100%)
Tempo gasto: 8.336006s


ok

----------------------------------------------------------------------
Ran 1 test in 9.031s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

- Índice com ocorrencias em memória secundária. Veja que, abaixo, que você pode ajustar o parametro de número de ocorrências em memória. Será muito útil para não gastar tanto tempo ao indexar o conteúdo da Wikipédia. 

In [17]:
import unittest
from index.performance_test import FilePerformanceTest,PerformanceTest
from index.structure import FileIndex

PerformanceTest.NUM_DOCS = 10
PerformanceTest.NUM_TERM_PER_DOC = 10000
FileIndex.TMP_OCCURRENCES_LIMIT = 10000

suite = unittest.TestLoader().loadTestsFromTestCase(FilePerformanceTest)
unittest.TextTestRunner(verbosity=2).run(suite)

Memoria usada: 4.27021 MB; Máximo 5.909258 MB
Indexando ocorrencia #100,000/100,000 (100%)
Tempo gasto: 9.052715s


ok

----------------------------------------------------------------------
Ran 1 test in 9.068s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [18]:
import unittest
from index.performance_test import FilePerformanceTest,PerformanceTest
from index.structure import FileIndex

PerformanceTest.NUM_DOCS = 10
PerformanceTest.NUM_TERM_PER_DOC = 50000
FileIndex.TMP_OCCURRENCES_LIMIT = 10000

suite = unittest.TestLoader().loadTestsFromTestCase(FilePerformanceTest)
unittest.TextTestRunner(verbosity=2).run(suite)

Memoria usada: 4.278269 MB; Máximo 5.921208 MB
Indexando ocorrencia #500,000/500,000 (100%)
Tempo gasto: 233.691548s


ok

----------------------------------------------------------------------
Ran 1 test in 233.707s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

Logo após executado este teste, você deverá usar a biblioteca [JSON](https://docs.python.org/3/library/json.html) ou [Pickle](https://docs.python.org/3/library/pickle.html) para armazenar o vocabulário. Com isso, crie um método de leitura do FileIndex e de escrita. O método de leitura deverá ser um método estatico que retorna um objeto da classe indice previamente criado.

## Indexador de HTML 

Agora, você irá alterar o arquivo `indexer.py` para [preprocessar conteúdo HTML](https://docs.google.com/presentation/d/1C22jQWIYobiqMx8SmP1y2lr1uSlvJSu3ayu5lXC5d8A/edit?usp=sharing) e depois indexá-lo. Com isso, você poderá usá-lo para indexação das páginas HTML, como os da Wikipédia. A classe `Cleaner` será responsável pelo preprocessamento e a HTMLIndexer, para a indexação.

In [19]:
!pip install nltk

You should consider upgrading via the '/Users/jonathan.candido/miniforge3/bin/python3.9 -m pip install --upgrade pip' command.


In [1]:
from index.indexer import *
#importamos o módulo structure
#novamente para não precisar de executar o código do início da tarefa
from index.structure import *

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jonathan.candido/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**Atividade 11 - Limpeza dos dados com a classe Cleaner: ** A classe `Cleaner` é responsável por preprocessar o conteúdo HTML para que ele esteja preparado para indexação. Essa classe tem alguns _flags_ para definir se algum tipo de processamento opcional será feito (por exemplo, _stemming_ e remoção de _stopwords_). Para isso, você deverá implementar pequenos métodos para fazer a limpeza. Esses códigos são pequenos pois temos lindas APIs para nos ajudar 💕. Você irá fazer o processamento básico e, se quiser, pode melhorar a implementação criando exceções na remoção de acentos e não retirando maiúsculas e minúsculas de certas palavras e unindo palavras compostas, por exemplo.

Para cada tarefa, há um método para ser criado a seguir os testes iniciais serão feitos aqui no Jupyter. Não esqueça de reiniciar o kernel sempre que alterar algo no código. Logo após, haverá um [teste de integração](https://en.wikipedia.org/wiki/Integration_testing) para avaliar a indexação como um todo.

- **Transformação de HTML para texto: ** Na limpeza dos dados, iremos remover tudo que não será indexado - ou seja, o código HTML. Para isso, iremos implementar o método `html_to_plain_text` que transformará o HTML em texto corrido. Você pode utilizar o [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) para isso e o método get_text. 

In [2]:
cleaner_test = Cleaner(stop_words_file="stopwords.txt",
                        language="portuguese",
                        perform_stop_words_removal=True,
                        perform_accents_removal=True,
                        perform_stemming=True)
cleaner_test.html_to_plain_text("&copy; oi! Meu nome é <strong>Hasan</strong>")
#esperado: '© oi! Meu nome é Hasan'

'© oi! Meu nome é Hasan'

- **Verifica se é stopword**: O método `is_stopword` retorna verdadeiro se uma palavra, passada como parâmetro, é stopword. Para isso, você irá usar o atributo `set_stop_words`. Este atibuto foi inicializado com um conjunto de stopwords de um arquivo. Esse arquivo, para testes, tem poucas stopwords. 
    

In [3]:
print(f"{cleaner_test.is_stop_word('japão')}, esperado: False")
print(f"{cleaner_test.is_stop_word('cama')}, esperado: False")
print(f"{cleaner_test.is_stop_word('é')}, esperado: True")


False, esperado: False
False, esperado: False
True, esperado: True


- **Stemming**: você deverá implementar o método word_stem para ralizar o stemming. Você deverá usar a [classe SnowballStemmer da API NLTK](https://www.nltk.org/howto/stem.html). Um objeto dessa classe já está instanciado no atributo `stemmer`. 

In [4]:
print(f"{cleaner_test.word_stem('verdade')}, esperado: verdad")
print(f"{cleaner_test.word_stem('estudante')}, esperado: estud")
print(f"{cleaner_test.word_stem('amado')}, esperado: amad")


verdad, esperado: verdad
estud, esperado: estud
amad, esperado: amad


- **Remoção de acentos:** Iremos fazer de uma forma bem simples a remoção de acentos: aplicando uma tabela de substituição de caracteres. Para isso, você deverá criar uma [tabela de tradução](https://docs.python.org/3.3/library/stdtypes.html?highlight=maketrans#str.maketrans) no atributo `accents_translation_table` baseando-se nas variáveis `in_table` e `out_table` também presentes no construtor (substitua o None).

In [5]:
print(f"{cleaner_test.remove_accents('canção')}, esperado: cancao")
print(f"{cleaner_test.remove_accents('elétrico')}, esperado: eletrico")
print(f"{cleaner_test.remove_accents('amado')}, esperado: amado")


cancao, esperado: cancao
eletrico, esperado: eletrico
amado, esperado: amado


Agora você irá fazer o método `preprocess_word` ele irá receber como parametro uma palavra e irá verificar se é uma palavra válida de ser indexada. Caso não seja, retornará None. Caso contrário, irá retornar a palvra preprocessada. Uma palavra válida a ser indexada é aquela que não é pontuação e não é stopword (caso `perform_stop_words_removal = True`). Para que seja feito o preprocessamento você deverá: transformar o texto para minúsculas, remover acento (se `perform_accents_removal=True`), fazer o stemming (se `perform_stemming = True`).

**Atividade 12 - Classe HTMLIndexer - método text_word_count: **  Você deverá implementar o método `text_word_count`, a partir de um texto. Esse método retorna um dicionário em que, para cada palavra no texto, será apresentado sua frequência. Considere que o texto já está limpo e é necessário fazer apenas o processamento das palavras.

Para isso, você deverá:  dividir o texto em tokens (que, no nosso caso, são as palavras e pontuações); preprocessar cada palavra usando o `HTMLIndexer.cleaner`; e, se for uma palavra válida, contabilizá-la. Para isso, será necessário [o método word_tokenize da API NLTK](https://kite.com/python/docs/nltk.word_tokenize)

In [6]:
index = HashIndex()
indexador_teste = HTMLIndexer(index)
indexador_teste.text_word_count("Olá! Qual é o dado dado que precisa?")
#esperado:
#{'dad': 2, 'o': 1, 'ola': 1, 'precis': 1, 'qual': 1, 'que': 1}

{'ola': 1, 'qual': 1, 'o': 1, 'dad': 2, 'que': 1, 'precis': 1}

**Atividade 13 - método index_text: ** Implemente o método `index_text` que deverá (1) converter o HTML para texto simples usando `HTMLIndexer.cleaner`; (2) converter o texto em um dicionário de ocorrencias de palavras com sua frequencia (metodo da atividade 12); e (3) indexar cada palavra deste dicionário.

In [ ]:

index = HashIndex()
indexador_teste = HTMLIndexer(index)
#o HTML está mal formado de propósito ;)
indexador_teste.index_text(10,"<strong>Ol&aacute;! </str> Quais são os dados que precisará?")

indexador_teste.index.dic_index

Esperado:
<pre>
{'dad': [(term_id:4 doc: 10 freq: 1)],
 'ola': [(term_id:0 doc: 10 freq: 1)],
 'os': [(term_id:3 doc: 10 freq: 1)],
 'precis': [(term_id:6 doc: 10 freq: 1)],
'qua': [(term_id:1 doc: 10 freq: 1)],
 'que': [(term_id:5 doc: 10 freq: 1)],
 'sao': [(term_id:2 doc: 10 freq: 1)]}
</pre>

**Atividade 14: Indexação de um diretorio com subdiretorios** Você deverá implementar o método `index_text_dir` que, dado um diretorio, navega em todos os seus subdiretórios e indexa todos os arquivos HTMLs. Considere que os arquivos sejam sempre nomeados pelo seu ID. Veja o exemplo em `doc_test`. Logo após, execute o teste unitário para ver a corretude do seu indexador.

In [ ]:
!python3 -m index.indexer_test IndexerTest.test_indexer

Agora, para fazer a especificação do projeto, você deve baixar o dataset da Wikipédia e indexá-lo. Você deve também achar um arquivo de stopwords com mais termos aos que feito neste teste.

In [1]:
from index.indexer import *
#importamos o módulo structure
#novamente para não precisar de executar o código do início da tarefa
from index.structure import *

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jonathan.candido/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
## lista de stopwords retirada de: https://gist.github.com/alopes/5358189
cleaner_test = Cleaner(stop_words_file="./wiki/stopwords.txt",
                        language="portuguese",
                        perform_stop_words_removal=True,
                        perform_accents_removal=True,
                        perform_stemming=True)
print(cleaner_test.set_stop_words)


{'das', 'estivermos', 'qual', 'houveram', 'houveríamos', 'nossa', 'seus', 'lhes', 'teve', 'por', 'hei', 'estivéramos', 'houve', 'houvéramos', 'houvessem', 'você', 'do', 'te', 'aquele', 'nós', 'é', 'num', 'estiverem', 'hajamos', 'houveria', 'mais', 'tínhamos', 'estávamos', 'foram', 'seriam', 'era', 'às', 'esta', 'as', 'estou', 'houvermos', 'fui', 'está', 'tiveram', 'tenhamos', 'pelo', 'os', 'com', 'deles', 'pelas', 'sejam', 'estão', 'aquelas', 'nossos', 'tém', 'isto', 'já', 'tiver', 'aquela', 'não', 'pelos', 'dele', 'ele', 'tivemos', 'tinham', 'tenham', 'numa', 'tu', 'esses', 'tenho', 'houvera', 'na', 'ela', 'foi', 'de', 'estava', 'teremos', 'sem', 'tenha', 'entre', 'até', 'nas', 'aquilo', 'teu', 'aqueles', 'esteja', 'forem', 'tivera', 'houveriam', 'o', 'fosse', 'esse', 'seríamos', 'pela', 'fomos', 'eram', 'estivemos', 'tivéssemos', 'houverão', 'tivessem', 'vos', 'eu', 'eles', 'meus', 'essa', 'ou', 'teus', 'se', 'dela', 'tivermos', 'seria', 'um', 'terei', 'houveremos', 'estiveram', 'hav

In [4]:
from index.indexer import *
from index.structure import *
import tracemalloc
time_first = datetime.now()
obj_index = FileIndex()
html_indexer = HTMLIndexer(obj_index)
tracemalloc.start()
html_indexer.index_text_dir("index/wiki/wikiSample")
current, peak = tracemalloc.get_traced_memory()            
time_end = datetime.now()
tempo_gasto = time_end-time_first 
print(f"Memoria usada: {current / 10**6:,} MB; Máximo {peak / 10**6:,} MB")
print(f"Finalizado:{tempo_gasto.total_seconds()}")

KeyboardInterrupt: 

In [2]:
a = 0
soma = 0
with open("times.txt","r",encoding="utf-8") as file:
    for line in file.readlines():
        a+=1
        soma+=  float(line.split(":")[1])

    print(f"Qtd arquivos: {a} Soma total: {soma} media:{(soma/a)} segundos")

Qtd arquivos: 1 Soma total: 0.04697 media:0.04697 segundos


In [2]:
#Recuperando o indice
from index.indexer import *
from index.structure import *
import pickle
with open("occur_index_0.idx","rb") as idx_file:
    obj_index = FileIndex()
    t = obj_index.next_from_file(idx_file)
    while t != None:
        print(t)
        t = obj_index.next_from_file(idx_file)

(term_id:0 doc: 1000 freq: 4)
(term_id:1 doc: 1000 freq: 11)
(term_id:2 doc: 1000 freq: 2)
(term_id:3 doc: 1000 freq: 5)
(term_id:4 doc: 1000 freq: 6)
(term_id:5 doc: 1000 freq: 3)
(term_id:6 doc: 1000 freq: 4)
(term_id:7 doc: 1000 freq: 2)
(term_id:8 doc: 1000 freq: 3)
(term_id:9 doc: 1000 freq: 2)
(term_id:10 doc: 1000 freq: 5)
(term_id:11 doc: 1000 freq: 1)
(term_id:12 doc: 1000 freq: 7)
(term_id:13 doc: 1000 freq: 8)
(term_id:14 doc: 1000 freq: 1)
(term_id:15 doc: 1000 freq: 4)
(term_id:16 doc: 1000 freq: 28)
(term_id:17 doc: 1000 freq: 4)
(term_id:18 doc: 1000 freq: 16)
(term_id:19 doc: 1000 freq: 2)
(term_id:20 doc: 1000 freq: 4)
(term_id:21 doc: 1000 freq: 1)
(term_id:22 doc: 1000 freq: 1)
(term_id:23 doc: 1000 freq: 1)
(term_id:24 doc: 1000 freq: 13)
(term_id:25 doc: 1000 freq: 1)
(term_id:26 doc: 1000 freq: 53)
(term_id:27 doc: 1000 freq: 9)
(term_id:28 doc: 1000 freq: 2)
(term_id:29 doc: 1000 freq: 1)
(term_id:30 doc: 1000 freq: 18)
(term_id:31 doc: 1000 freq: 1)
(term_id:32 